In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from keras.layers import LeakyReLU
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Define a custum loss function
def mse_weighted_loss(y_true, y_pred):
  loss = tf.reduce_mean(tf.square(y_true - y_pred)*(4.0*tf.abs(y_true)+0.5))
  return loss

In [3]:
# Import the trained model
metamorphosisNN_model = load_model("metamorphosisNN.keras",
                        custom_objects={"LeakyReLU": LeakyReLU,
                                        "mse_weighted_loss": mse_weighted_loss})

# Function to override Batch Norm behavior
class MonteCarloModel(tf.keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def call(self, inputs, training=None):
        outputs = inputs
        for layer in self.model.layers:
            if isinstance(layer, keras.layers.BatchNormalization):
                # Force BatchNorm to use moving averages from training
                outputs = layer(outputs, training=False)
            else:
                # Apply other layers normally with `training=True` for dropout
                outputs = layer(outputs, training=training)
        return outputs

# Wrap the trained model
metamorphosisNN = MonteCarloModel(metamorphosisNN_model)

# Function to calculate the MC predictions
def monte_carlo_predictions(mc_model, X, n_simulations=100):
    predictions = np.array([mc_model(X, training=True).numpy() for _ in range(n_simulations)])
    predictions = pd.DataFrame(predictions.squeeze().T)
    return predictions

In [28]:
# Mock data:
mock_data = pd.DataFrame(data = [
    [10.728 ,12.331,	9.502,	7.804,	7.178,	6.904,	6.698,	6.606], # HD 18143C
    [10.053, 11.523,	8.866,	7.231,	6.701,	6.487,	6.346,	6.160]  # HD 20727B
    ],
    columns=["M_G", "M_BP", "M_RP", "M_J", "M_H", "M_K", "M_W1", "M_W2"])

predictions = monte_carlo_predictions(metamorphosisNN, mock_data, n_simulations=100)

print("--------------------------")
print("Results:")
print("--------------------------")
for i in range(len(predictions)):
  print("[Fe/H] = {0:5.2f} +- {1:3.2f} dex".format(predictions.mean(axis=1)[i], predictions.std(axis=1)[i]))
print("--------------------------")

--------------------------
Results:
--------------------------
[Fe/H] =  0.15 +- 0.04 dex
[Fe/H] = -0.28 +- 0.08 dex
--------------------------
